In [2]:
from graphviz import Digraph

COLS = ["#87193dD9", "#80A1D4", "#893168D9", "#75C9C8", "#DED9E2"]

"""
Graphviz DAG for Cepheid-based H0 inference.
Nodes represent priors, deterministic calculations, likelihoods, and observed data.
Cluster highlights the velocity field and Hubble flow submodel.
"""
dot = Digraph(format='pdf')
dot.attr(rankdir='TB')                 # top-to-bottom
dot.attr(splines='true')
dot.attr(fontsize='30', nodesep='0.6', ranksep='1.0')
dot.attr(concentrate='true')

# Use LaTeX-style serif font
dot.attr(fontname='CMU Serif')
dot.attr('node', fontname='CMU Serif', fontsize='30')
dot.attr('edge', fontname='CMU Serif', fontsize='30', arrowsize='2')


# --- Priors (merged μ_NGC4258, μ_LMC, μ_M31) ---
prior_nodes = {
    'H0': '<H<SUB>0</SUB>>',
    'MW_bW_ZW': '<M<SUB>W</SUB>, b<SUB>W</SUB>,<BR/>Z<SUB>W</SUB>, Δ<SUB>ZP</SUB>>',
    'sigma_v': '<σ<SUB>v</SUB>>',
    'beta': 'β',
    'Vext': '<V<SUB>ext</SUB>>',
    'galaxy_bias': '<Galaxy<BR/>bias>',
    'mu_host': 'Host\ndistances',
    'mu_anchors': '<μ<SUB>NGC4258</SUB>,<BR/>μ<SUB>LMC</SUB>, μ<SUB>M31</SUB>>',
}
for var, label in prior_nodes.items():
    dot.node(var, label, shape='ellipse', style='filled', fillcolor=COLS[1])

# --- External data ---
dot.node('density_field', 'Density\nfield', shape='ellipse', style='filled', fillcolor=COLS[-1])
dot.node('velocity_field', 'Velocity\nfield', shape='ellipse', style='filled', fillcolor=COLS[-1])

# --- Deterministic nodes ---
deterministic_nodes = [
    ('Vpec', '<Host<BR/>V<SUB>pec</SUB>>'),
    ('cz_pred', '<Host<BR/>z<SUB>pred</SUB>>'),
    ('mu_cepheid', '<Cepheid<BR/>μ>'),
    ('mag_pred', '<Cepheid<BR/>m<SUB>pred</SUB>>'),
]
for var, label in deterministic_nodes:
    dot.node(var, label, shape='ellipse', style='filled', fillcolor=COLS[3])

# --- Likelihood nodes (merged anchor cal.) ---
likelihood_nodes = [
    ('anchor_ll', '<NGC4258, LMC cal.<BR/>| μ<SUB>NGC4258</SUB>, μ<SUB>LMC</SUB>>'),
    ('redshift_ll', '<Host<BR/>z<SUB>CMB</SUB> | z<SUB>pred</SUB>>'),
    ('cepheid_ll', '<Cepheid<BR/>m<SUB>obs</SUB> | m<SUB>pred</SUB>>'),
    ('MW_ll', '<MW CPLR cal.<BR/>| M<SUB>W</SUB>>'),
]
for var, label in likelihood_nodes:
    dot.node(var, label, shape='ellipse', style='filled', fillcolor=COLS[2])

# --- Observed data nodes (including merged logP+O/H, and merged NGC/LMC cal.) ---
data_nodes = [
    ('logP_OH', 'Cepheid\nlog P, O/H'),
    ('cz_obs', '<Host<BR/>z<SUB>CMB</SUB>>'),
    ('mag_obs', '<Cepheid<BR/>m<SUB>obs</SUB>>'),
    ('anchors_geo', 'NGC4258,\nLMC cal.'),
    ('MW_data', 'MW CPLR\ncal.'),
]
for var, label in data_nodes:
    dot.node(var, label, shape='ellipse', style='filled', fillcolor=COLS[-1])

# --- Cluster for velocity / H0 submodel ---
with dot.subgraph(name='cluster_velocity_block') as c:
    c.attr(label="Hubble constant inference",
           color='black',
           style='dashed', fontsize='36',
           labelloc='b')
    c.node('velocity_field')
    c.node('beta')
    c.node('Vext')
    c.node('H0')
    c.node('Vpec')
    c.node('cz_pred')
    c.node('redshift_ll')
    c.node('sigma_v')
    c.node('cz_obs')

# --- Graph edges ---
dot.edge('mu_host', 'Vpec')
dot.edge('beta', 'Vpec')
dot.edge('Vext', 'Vpec')  # V_ext contributes to host Vpec
dot.edge('velocity_field', 'Vpec')

dot.edge('density_field', 'mu_host')
dot.edge('density_field', 'mu_anchors')

dot.edge('galaxy_bias', 'mu_host')
dot.edge('galaxy_bias', 'mu_anchors')

dot.edge('mu_host', 'cz_pred')
dot.edge('Vpec', 'cz_pred')
dot.edge('H0', 'cz_pred')
dot.edge('cz_pred', 'redshift_ll')
dot.edge('sigma_v', 'redshift_ll')

# Anchors merged edges
dot.edge('mu_anchors', 'anchor_ll')
dot.edge('anchors_geo', 'anchor_ll')

dot.edge('mu_host', 'mu_cepheid')
dot.edge('mu_anchors', 'mu_cepheid')

dot.edge('mu_cepheid', 'mag_pred')
dot.edge('MW_bW_ZW', 'mag_pred')
dot.edge('logP_OH', 'mag_pred')

dot.edge('mag_pred', 'cepheid_ll')

# Observed data to likelihoods
dot.edge('cz_obs', 'redshift_ll')
dot.edge('mag_obs', 'cepheid_ll')

# MW constraint on merged prior node
dot.edge('MW_data', 'MW_ll')
dot.edge('MW_bW_ZW', 'MW_ll')

# --- Render ---
dot.render('/Users/rstiskalek/Downloads/CH0_DAG', view=True)

'/Users/rstiskalek/Downloads/CH0_DAG.pdf'